# CONFIGURAÇÃO

In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from datetime import date

In [2]:
data_atual = date.today().strftime('%d-%m-%Y')
FILENAME = f"{data_atual}-autismo-"

08-06-2022-autismo-


# LEITURA DE ARQUIVOS

In [3]:
conf = SparkConf()
jars = [
    'org.apache.hadoop:hadoop-aws:3.2.2', 'com.microsoft.azure:spark-mssql-connector_2.12:1.2.0', 
    'com.microsoft.sqlserver:mssql-jdbc:7.0.0.jre8',
    'com.databricks:spark-xml_2.12:0.14.0'
]
conf.set('spark.jars.packages', ','.join(jars))
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')

In [4]:
spark = SparkSession.builder\
    .appName('load-azure')\
    .config(conf=conf)\
    .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.microsoft.azure#spark-mssql-connector_2.12 added as a dependency
com.microsoft.sqlserver#mssql-jdbc added as a dependency
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-06d3cda3-fae7-4bf8-b242-39d2d61ad85d;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
	found com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 in central
	found com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 in central
	found com.databricks#spark-xml_2.12;0.14.0 in central
	found commons-io#commons-io;2.8.0 in central
	found org.glassfish.jaxb#txw2;2.3.4 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.2.5 in central
:: resolution report :: resolve 569ms :: artifacts dl 20ms
	:: modules in use:
	com.ama

In [5]:
person_data = spark.read.csv('s3a://sprint-3-luiza-grupo-09-bucket-brutos/03-06-2022-autismo-csv.csv', header=True)
print(person_data)

22/06/08 02:34:26 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


DataFrame[id_paciente: string, idade_atual: string, idade_descoberta: string, genero: string, grau: string, sensibilidade_sentidos: string, agressivo: string, hiperativo: string, movimentos_repetitivos: string, baixa_concentracao: string, hiperfoco: string, necessidade_rotina: string, dificuldade_imaginacao: string, introvertido: string, tipo_autismo: string, tipo_autismo_descricao: string]


In [6]:
make_data = spark.read.json('s3a://sprint-3-luiza-grupo-09-bucket-brutos/03-06-2022-autismo-json.json')
print(make_data)

DataFrame[classe: string, classe_descricao: string, empregado: boolean, id_renda_paciente: bigint, per_capita: double, quantidade_familia: bigint, total_salario: bigint]


In [7]:
region_data =  spark.read.format("com.databricks.spark.xml")\
    .option("rootTag", "data") \
    .option("rowTag", "row") \
    .load("s3a://sprint-3-luiza-grupo-09-bucket-brutos/LE-ESSA-DESGRACA.xml")\

In [ ]:
person_data.show(truncate=False, vertical=True)

In [ ]:
make_data.show(truncate=False, vertical=True)

In [ ]:
region_data.show(truncate=False, vertical=True)

# MANIPULACAO

In [ ]:
data_referencing = (
    person_data
        .join(make_data, person_data.id_paciente == make_data.id_renda_paciente)\
        .join(region_data, person_data.id_paciente == region_data.id_regiao_paciente)\
        .select(
            person_data.id_paciente,person_data.idade_atual,person_data.idade_descoberta,person_data.genero,
            person_data.grau,person_data.sensibilidade_sentidos,person_data.agressivo,person_data.hiperativo,
            person_data.movimentos_repetitivos,person_data.baixa_concentracao,person_data.hiperfoco,person_data.necessidade_rotina,
            person_data.dificuldade_imaginacao,person_data.introvertido,person_data.tipo_autismo,person_data.tipo_autismo_descricao,
            make_data.classe,make_data.classe_descricao,make_data.empregado,make_data.per_capita,make_data.quantidade_familia,
            make_data.total_salario,region_data.PIB_nacional,region_data.PIB_percent_BR,region_data.PIB_regional,
            region_data.area_regiao,region_data.densidade_demografica_regiao,region_data.gentilico_regiao,region_data.id_regiao_paciente,
            region_data.pais,region_data.populacao_regiao,region_data.raca_predominante_regiao,region_data.regiao
        )
)

In [ ]:
data_referencing.printSchema()

In [ ]:
data_referencing.show(truncate=False, vertical=True).to_pandas()

# SALVANDO DADOS NO BUCKET-PROCESSADOS

In [13]:
data_referencing.write.csv(f's3a://sprint-3-luiza-grupo-09-bucket-processados/{FILENAME}_dados_agrupados.csv', header=True)

# CONEXÃO COM AZURE

In [8]:
server = "teste-sprint3.database.windows.net"
port = 1433
db = "teste1000"
username = "urubu100"
password = "Urubu1@@"

url = f'jdbc:sqlserver://{server}:1433;database={db};'
print(url)

jdbc:sqlserver://teste-sprint3.database.windows.net:1433;database=teste1000;


In [10]:
person_data.write\
    .format("com.microsoft.sqlserver.jdbc.spark") \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .mode("overwrite") \
    .option('tableLock', True) \
    .option("url", url) \
    .option("dbtable", 'paciente') \
    .option("user", username) \
    .option("password", password) \
    .save()

In [11]:
make_data.write\
    .format("com.microsoft.sqlserver.jdbc.spark") \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .mode("overwrite") \
    .option('tableLock', True) \
    .option("url", url) \
    .option("dbtable", 'renda') \
    .option("user", username) \
    .option("password", password) \
    .save()

In [12]:
region_data.write\
    .format("com.microsoft.sqlserver.jdbc.spark") \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .mode("overwrite") \
    .option('tableLock', True) \
    .option("url", url) \
    .option("dbtable", 'localidade') \
    .option("user", username) \
    .option("password", password) \
    .save()